In [48]:
## Library
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime

In [49]:
#URL
URL = "https://www.blockchain.com/btc/unconfirmed-transactions"

def searchHTML(URL):
    #Request HTML & beautify it
    page = requests.get(URL)
    soup = bs(page.content)
    
    #Find right data
    results_numbers = soup.find_all("span", {'class' : 'sc-1ryi78w-0 cILyoi sc-16b9dsl-1 ZwupP u3ufsr-0 eQTRKC'})
    results_hash = soup.find_all("a", {'class' : 'sc-1r996ns-0 fLwyDF sc-1tbyx6t-1 kCGMTY iklhnl-0 eEewhk d53qjk-0 ctEFcK'})
    
    return results_numbers, results_hash

def makeFrame(hash, time, BTC, USD):
    dataF = {'Hash' : hash, 'Time' : time, 'BTC-value' : BTC, 'USD-value' : USD}
    frame = pd.DataFrame(data=dataF)
    
    return frame

def makeArrays(numbers, hash):
    only_time = [numbers[i] for i in range(0, len(numbers), 3)]
    only_BTC = [numbers[i] for i in range(1, len(numbers), 3)]
    only_USD = [numbers[i] for i in range(2, len(numbers), 3)]
    only_hash = [hash[i] for i in range(0, len(hash), 1)]
    
    return only_hash, only_time, only_BTC, only_USD

In [50]:
def fetchCurrent(URL):
    #HTML-doorzoeken en alles ophalen
    numbers, hash = searchHTML(URL)

    #Make arrays of only required values
    only_hash, only_time, only_BTC, only_USD = makeArrays(numbers, hash)

    #Make dataframe
    frame = makeFrame(only_hash, only_time, only_BTC, only_USD)

    #Tells current time
    print(f"{datetime.now().hour}:{datetime.now().minute}")

    #Print Dataframe
    print()
    print(frame)

fetchCurrent(URL)

23:31

                                                 Hash     Time  \
0   [9ea9ac8ecdcd2ac502b3bc3302e768477da47ba89be73...  [22:31]   
1   [170b7a4e2506173c6df33b51a8c0ebeef014edf890660...  [22:31]   
2   [223370406a27976d0e2f62ad4874c22c06fbd6ecac505...  [22:31]   
3   [948aae4c557a07e82bf69c0bec87bd55eeac595238a35...  [22:31]   
4   [8f6bd7718cee69f29947c8c037d549a7ccd6ec9565694...  [22:31]   
5   [8d57dffb6ef9c616a24aed61d142fe9e1828edb3e4723...  [22:31]   
6   [0d0a5587aedeca48b915662437621b82b1756d8d8ba8a...  [22:31]   
7   [75e55b2f649f1eb0dd74c41b756ff220b24bc8ff4cee0...  [22:31]   
8   [d5d08070c3f76b7b65acadfbc8e495d65885d4253e0c5...  [22:31]   
9   [b9df36e350785320a0258d00d79118a61447f5dc56eda...  [22:31]   
10  [b5ed9e350305c11c501fcd135b910ec974fc009a3422e...  [22:31]   
11  [e239df8287f60c209e117e02979093f10eb81a8d7e3cc...  [22:31]   
12  [1a1e6eb5c2f7ad03a71fe2512519557fe1e706b596d47...  [22:31]   
13  [80f6cd27db47210327b092704aceba71d1102a772c359...  [22:31]   
14 